### word2vec을 이용한 단어 embedding

In [1]:
from gensim.models import Word2Vec
import re

In [2]:
# GPT야, simpsons 캐릭터 이름이 들어간 랜덤 문장 10개를 생성해줘

sentences = ["Homer Simpson forgot his lunch at home, so he had to buy a burger on his way to work.",
    "Marge was busy knitting a new sweater for Bart's upcoming school play.",
    "Lisa Simpson played a beautiful saxophone solo at the school concert.",
    "Mr. Burns secretly plotted another scheme from his office at the Springfield Nuclear Power Plant.",
    "Ned Flanders offered to help Homer fix the fence between their houses.",
    "Bart Simpson tried a new prank at school, but it didn't go as planned.",
    "Milhouse and Bart spent the afternoon playing video games and forgot to do their homework.",
    "Maggie Simpson's adorable giggle filled the room as she played with her toys.",
    "Apu had a busy day at the Kwik-E-Mart, dealing with a rush of customers.",
    "Krusty the Clown decided to change his show a bit to attract a new audience."]

In [3]:
# preprocessing
# get rid of stopwords, lower case

sentences = [re.sub(r"[.',]", "", sentence).lower().split(" ") for sentence in sentences]

In [4]:
sentences[0]

['homer',
 'simpson',
 'forgot',
 'his',
 'lunch',
 'at',
 'home',
 'so',
 'he',
 'had',
 'to',
 'buy',
 'a',
 'burger',
 'on',
 'his',
 'way',
 'to',
 'work']

In [6]:
# train word2vec

skip_gram = Word2Vec(sentences, vector_size=300, min_count=1, window=5, sg=1)

In [7]:
print("{} 의 vector representation : \n{}".format('homer', skip_gram.wv.get_vector(skip_gram.wv.key_to_index['homer'])))

homer 의 vector representation : 
[-1.66068680e-03 -4.20303462e-04  1.09255337e-03 -2.18484551e-03
 -3.23499087e-03 -3.12648294e-03  3.04949097e-03  1.87829603e-03
 -1.61115616e-03 -2.80928891e-03  4.52324050e-04  9.58630932e-04
 -4.38321120e-04  4.20039461e-04 -1.44900673e-03  1.57583959e-03
  5.10560756e-04  2.96067167e-03 -3.31212161e-03 -1.82445510e-03
 -3.01285670e-03 -1.11729649e-04 -2.61634029e-03  1.71959959e-03
 -2.11100094e-03 -2.03485345e-03  1.69092277e-03 -2.72677210e-03
  4.76851739e-04 -2.43410910e-03  3.28659941e-03  2.89322482e-03
  5.95899648e-04  1.94471062e-03  1.52419391e-03 -1.98301394e-03
  3.28957522e-03 -3.26667982e-03  2.68764189e-03  9.17330268e-04
 -9.88418004e-04 -1.19753159e-03  3.04961996e-03 -1.81994250e-03
  2.79433304e-03 -1.95553945e-03  2.79207341e-03 -1.48160951e-04
  2.66201468e-03 -1.02206389e-03  2.01950571e-03  2.95562134e-03
  8.11300066e-04  4.55215428e-04  1.65783032e-03  2.71425396e-03
  2.85594515e-03  2.84643611e-03  2.36571440e-03  2.69569

In [8]:
skip_gram.wv.most_similar("homer")

[('marge', 0.14081519842147827),
 ('offered', 0.13243569433689117),
 ('games', 0.12250108271837234),
 ('her', 0.11486156284809113),
 ('nuclear', 0.10569248348474503),
 ('do', 0.09913019835948944),
 ('toys', 0.0984482392668724),
 ('office', 0.0924413651227951),
 ('bart', 0.09009940177202225),
 ('way', 0.08802291750907898)]

직접 유사도 구해보기

In [9]:
homer_vector = skip_gram.wv.get_vector(skip_gram.wv.key_to_index['homer'])
marge_vector = skip_gram.wv.get_vector(skip_gram.wv.key_to_index['marge'])

In [12]:
# 유사도 계산하기 from scratch
import numpy as np
from numpy.linalg import norm

def cosine_similarity(vector_a: np.ndarray, vector_b: np.ndarray) -> float:
    """
    두 벡터간 cosine similarity를 계산
    
    Parameters
    ----------
    vector_a : np.ndarray
        The first input vector.
    vector_b : np.ndarray
        The second input vector.

    Returns
    -------
    float
        The cosine similarity between `vector_a` and `vector_b`, which is a value between -1 and 1.

    """

    dot_product = np.dot(vector_a, vector_b)
    norm_a = norm(vector_a)
    norm_b = norm(vector_b)
    similarity = dot_product / (norm_a * norm_b)
    return similarity

In [13]:
cosine_similarity(homer_vector, marge_vector)

0.14081518

### Simpsons dataset을 활용한 Word2Vec

![](https://images.edrawmax.com/what-is/simpsons-family-tree/example.png) <br>
출처 : https://images.edrawmax.com/what-is/simpsons-family-tree/example.png

In [14]:
from typing import Optional
import nltk
import re
import pandas as pd
import spacy

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/gieunkwak/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/gieunkwak/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
df = pd.read_csv('simpsons_dataset.csv')
df.shape

(158314, 2)

In [16]:
df.head()

,raw_character_text,spoken_words
0,Miss Hoover,"No, actually, it was a little of both. Sometim..."
1,Lisa Simpson,Where's Mr. Bergstrom?
2,Miss Hoover,I don't know. Although I'd sure like to talk t...
3,Lisa Simpson,That life is worth living.
4,Edna Krabappel-Flanders,The polls will be open from now until the end ...


In [17]:
df.isnull().sum()

raw_character_text    17814
spoken_words          26459
dtype: int64

In [28]:
df.loc[0, 'spoken_words']

"No, actually, it was a little of both. Sometimes when a disease is in all the magazines and all the news shows, it's only natural that you think you have it."

#### 데이터 전처리

In [20]:
# lemmatize and remove the stopwords and non-alphabetic characters for each line of dialogue

nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser'])

def cleaning(doc):
    """
    Cleans a spaCy Doc object by lemmatizing its tokens and removing stop words,
    then joins the remaining tokens into a single string if there are more than two tokens left.
    
    Parameters:
    ----------
    doc : spacy.tokens.Doc
        A spaCy Doc object containing the processed text.
    
    Returns:
    ----------
    Optional : str
        A string composed of the lemmatized, non-stop tokens separated by spaces,
        if the resulting list of tokens has more than two elements. Otherwise, returns None.
    """

    txt = [token.lemma_ for token in doc if not token.is_stop]
    if len(txt) > 2:
        return ' '.join(txt)

In [21]:
# only keep alphabets
cleaner = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['spoken_words'])

In [22]:
txt = [cleaning(doc) for doc in nlp.pipe(cleaner, batch_size=5000)]

In [23]:
txt[0]

'actually little disease magazine news show natural think'

In [25]:
# dataframe에 넣어서 null이 있는 대화는 삭제
# 주로 null은 특정 행동을 했지만 대화가 없었을 때임

df_clean = pd.DataFrame({'clean': txt})
df_clean = df_clean.dropna().drop_duplicates()
df_clean.shape

(85956, 1)

In [26]:
# 하나의 문장을 여러 단위의 단어로 분할
sentences = [s.split(' ') for s in df_clean['clean']]

In [27]:
len(sentences)

85956

In [28]:
sentences[0]

['actually',
 'little',
 'disease',
 'magazine',
 'news',
 'show',
 'natural',
 'think']

#### Word2Vec 모델 훈련

In [29]:
from gensim.models import Word2Vec

In [31]:
# help(Word2Vec)

- `window` : 문장 내에서 현재 단어와 예측 단어 사이의 최대 거리. ex) 타겟 단어의 왼쪽과 오른쪽 n번째 단어
- `vector_size` : 단어 벡터의 차원 수
- `min_count` : 이 값보다 총 절대 빈도수가 낮은 모든 단어를 무시함 - (2, 100)
- `sg` : 1은 skip-gram, 0은 CBOW method를 사용

In [32]:
# 모델 정의 하기
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     vector_size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007)

In [33]:
# 문장에 들어있는 각 단어들을 Word2Vec 모델이 인식할 수 있는 형태로 변환
w2v_model.build_vocab(sentences)

In [34]:
# 모델 훈련
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=100)

(19990485, 54001900)

In [36]:
help(w2v_model.train)

Help on method train in module gensim.models.word2vec:

train(corpus_iterable=None, corpus_file=None, total_examples=None, total_words=None, epochs=None, start_alpha=None, end_alpha=None, word_count=0, queue_factor=2, report_delay=1.0, compute_loss=False, callbacks=(), **kwargs) method of gensim.models.word2vec.Word2Vec instance
    Update the model's neural weights from a sequence of sentences.
    
    Notes
    -----
    To support linear learning-rate decay from (initial) `alpha` to `min_alpha`, and accurate
    progress-percentage logging, either `total_examples` (count of sentences) or `total_words` (count of
    raw words in sentences) **MUST** be provided. If `sentences` is the same corpus
    that was provided to :meth:`~gensim.models.word2vec.Word2Vec.build_vocab` earlier,
    you can simply use `total_examples=self.corpus_count`.
    
    Warnings
    --------
    To avoid common mistakes around the model's ability to do multiple training passes itself, an
    explicit `epoc

### 단어간 유사도 확인하기

In [38]:
# dir(w2v_model.wv)

- most_similar : 주어진 조건에 가장 적합한 단어 탐색
- similarity : 주어진 단어들의 유사도 계산
- doesnt_match : 주어진 단어들 중 가장 '덜 유사한' 단어

In [48]:
# help(w2v_model.wv.most_similar)

In [49]:
# help(w2v_model.wv.similarity)

In [39]:
w2v_model.wv.most_similar(positive=["homer"])

[('marge', 0.39361003041267395),
 ('son', 0.3050530254840851),
 ('simpson', 0.2926293909549713),
 ('bart', 0.28149035573005676),
 ('mr', 0.2747665047645569),
 ('husband', 0.2533870339393616),
 ('lisa', 0.24030055105686188),
 ('dad', 0.22668324410915375),
 ('smither', 0.22446727752685547),
 ('homie', 0.21520781517028809)]

In [40]:
w2v_model.wv.most_similar(positive=["bart"])

[('lisa', 0.4804288148880005),
 ('dad', 0.3908121585845947),
 ('boy', 0.35573628544807434),
 ('mom', 0.34492626786231995),
 ('son', 0.281699538230896),
 ('homer', 0.28149035573005676),
 ('milhouse', 0.2809959948062897),
 ('mother', 0.2730790376663208),
 ('maggie', 0.2726394832134247),
 ('child', 0.27139681577682495)]

- Woman : homer = ___ : marge

In [41]:
w2v_model.wv.most_similar(positive=["woman", "homer"], negative=["marge"], topn=3)

[('man', 0.2424401193857193),
 ('young', 0.22867713868618011),
 ('see', 0.2077772170305252)]

In [42]:
w2v_model.wv.most_similar(positive=["woman", "bart"], negative=["man"], topn=3)

[('mom', 0.2672269642353058),
 ('lisa', 0.26112234592437744),
 ('embarrassing', 0.2553335726261139)]

In [44]:
w2v_model.wv.doesnt_match(['bart', 'homer', 'marge'])

'bart'

In [46]:
w2v_model.wv.doesnt_match(['bart', 'lisa', 'marge'])

'marge'

### 단어 임베딩의 한계점

In [64]:
bank_vector = w2v_model.wv.get_vector(w2v_model.wv.key_to_index['bank'])

In [66]:
# bank_vector

In [51]:
# bank_vector

- 우리가 사용하는 모든 단어는 context에 따라 의미가 다르다
- 단어 embedding의 경우 이런 유연성을 확보하지 못 함
    - 배를 깎아 먹었다 / 배가 고프다 / 배 멀미를 하다

In [10]:
# bank_vector

### sentence embeddings

In [67]:
from transformers import BertTokenizer, BertModel
import torch

In [68]:
# pre-trained model tokenizer와 and bert model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased') # smaller & uncased model
model = BertModel.from_pretrained('bert-base-uncased')

In [70]:
# bank가 들어간 유사한 문장 두 개
sentence1 = "I deposited money at the bank."
sentence2 = "The ducks swam to the river bank."

In [71]:
# 문장을 BERT가 인식할 수 있는 형태로 Tokenize
encoded_input1 = tokenizer(sentence1, return_tensors='pt') # pytorch
encoded_input2 = tokenizer(sentence2, return_tensors='pt')

In [72]:
encoded_input1

{'input_ids': tensor([[  101,  1045, 14140,  2769,  2012,  1996,  2924,  1012,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1]])}

- `input_ids` : 각 단어별로 매핑된 key. 101은 문장의 시작을, 102는 문장의 끝을 의미
- `token_type_ids` : 문장 번호
- `attention_mask` : attention을 가져야 하는 단어는 1, 그렇지 않은 단어는 0. (만약 input이 실제 단어들이라면 1)

In [74]:
# embedding 생성!
with torch.no_grad():
    output1 = model(**encoded_input1)
    output2 = model(**encoded_input2)

In [76]:
# embedding 내에서 bank라는 단어 찾아오기 (문장의 5번째에 있는 단어)
bank_embedding_sentence1 = output1.last_hidden_state[0, 5, :]
bank_embedding_sentence2 = output2.last_hidden_state[0, 5, :]

In [80]:
# cosine similarity 계산

similarity = cosine_similarity(bank_embedding_sentence1, bank_embedding_sentence2)
# print("Embedding for 'bank' in sentence 1:", bank_embedding_sentence1)
# print("Embedding for 'bank' in sentence 2:", bank_embedding_sentence2)
print("Cosine similarity between the two embeddings:", similarity)

Cosine similarity between the two embeddings: 0.59224105


--END--